# The Sound of a Transiting Exoplanet

In [ ]:
import matplotlib.pyplot as plt

## Data

In [ ]:
from astropy.utils.data import get_pkg_data_filename
from astropy.timeseries import TimeSeries

import numpy as np
from astropy import units as u
from astropy.timeseries import BoxLeastSquares
from astropy.stats import sigma_clipped_stats
from astropy.timeseries import aggregate_downsample
import matplotlib.pyplot as plt
import pandas as pd

filename = get_pkg_data_filename('timeseries/kplr010666592-2009131110544_slc.fits')  
ts = TimeSeries.read(filename, format='kepler.fits') 

periodogram = BoxLeastSquares.from_timeseries(ts, 'sap_flux')  

results = periodogram.autopower(0.2 * u.day)  
best = np.argmax(results.power)  
period = results.period[best]   
transit_time = results.transit_time[best]  

mean, median, stddev = sigma_clipped_stats(ts['sap_flux'])  
ts['sap_flux_norm'] = ts['sap_flux'] / median  
ts_binned = aggregate_downsample(ts, time_bin_size=0.02 * u.day)  

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(ts.time.jd, ts['sap_flux'], 'k.', markersize=1)
plt.xlabel('Time (days)')
plt.ylabel('SAP Flux (e-/s)')

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(ts_binned.time_bin_start.jd, ts_binned['sap_flux_norm'], 'k.', markersize=2)
plt.xlabel('Time (days)')
plt.ylabel('SAP Flux (e-/s)')
plt.plot(ts_binned.time_bin_start.jd, ts_binned['sap_flux_norm'], 'r-', drawstyle='steps-post')
plt.xlabel('Time (days)')
plt.ylabel('Normalized flux')

## Sonification

In [ ]:
from scisonify.core.soundmaps import DiscreteNoteBins

In [ ]:
smap = DiscreteNoteBins.from_key("C:dor", octave_range=(3, 4))

In [ ]:
import scisonify.pandas

In [ ]:
import pandas as pd

sap_flux_norm_binned = pd.Series(ts_binned['sap_flux_norm'].value)
time = ts_binned.time_bin_start.jd

In [ ]:
sap_flux_norm_binned.sonify(smap=smap, note_length = 0.10)

In [ ]:
sap_flux_norm_binned.sonify.plot(x=time, ms=1, figsize=(20, 10), title="Normalized & Binned SAP Flux as Musical Notes", xlabel="Julian Date")